In [ ]:
from vsts.vss_connection import VssConnection
from msrest.authentication import BasicAuthentication
from vsts.work_item_tracking.v4_1.models.wiql import Wiql
from types import SimpleNamespace

import argparse, os

In [ ]:
def area(path):
    split = path.split('\\')
    sub_area = split[-1]
    if len(split)>1:
        main_area = split[1]
    else:
        main_area = split[0]
    return (main_area, sub_area)

In [1]:
import csv
import time
from datetime import datetime, date, timedelta
import argparse, os
from devops_connection import get_query_papercuts, get_connstrings, get_devopsconnection, get_query_ids, get_work_items
from utils import html2text
import argparse, os



In [2]:
project = get_connstrings()


conn = get_devopsconnection(project)

In [48]:
bugids = get_query_ids(conn, get_query_papercuts())
work_items = get_work_items(conn, bugids)
print("Papercuts: ", len(work_items))

Executing query: 
    SELECT
        [System.Id],
        [System.CreatedDate],
        [System.Title],
        [System.CreatedBy],
        [System.State],
        [Microsoft.VSTS.Common.Priority],
        [System.AreaPath],
        [System.AssignedTo],
        [System.Tags],
        [System.IterationPath]
    FROM workitems
    WHERE
        [System.TeamProject] = 'Vienna'
        AND (
            [System.WorkItemType] <> ''
            AND [System.Tags] CONTAINS 'Papercuts'
            OR [System.Tags] CONTAINS 'papercut'
            OR [System.Tags] CONTAINS 'WS20GA'
        )
    ORDER BY [System.AreaPath]
    
Papercuts:  450


In [49]:
IMPORT_FIELDS = ['id', 'rev', 'AreaPath', 'TeamProject', 'IterationPath', 'WorkItemType', 'State', 'AssignedTo', 'CreatedDate', 'CreatedBy', 'ChangedDate', 'ChangedBy', 'CommentCount', 'Title', 'StateChangeDate', 'Priority', 'Tags', ]


In [50]:
def area(path):
    split = path.split('\\')
    sub_area = split[-1]
    if len(split)>1:
        main_area = split[1]
    else:
        main_area = split[0]
    return (main_area, sub_area)

def format_name(user):
    if user is None:
        return None
    else:
        return user['displayName'] + " <" + user['uniqueName'] + ">"



In [51]:


from datetime import datetime
today = datetime.utcnow().date().isoformat()
work_items_array = []
count = 0
for wi in work_items:
    if (count % 100) == 0:
        print(count, wi.id)
    count+=1
    wi_dict = wi.as_dict()
    fields = wi_dict['fields']
    del wi_dict['fields']
    wi_dict.update(fields)
    wi_dict = {key.split('.')[-1]: wi_dict[key] for key in wi_dict if key.split('.')[-1] in IMPORT_FIELDS}
    wi_dict['import_date'] = today
    wi_dict['weburl'] = f"{project['url']}/{project['id']}/_workitems/edit/{wi.id}"
    wi_dict['CreatedBy'] = format_name(wi_dict['CreatedBy'])
    wi_dict['ChangedBy'] = format_name(wi_dict['ChangedBy'])
    if 'AssignedTo' in wi_dict:
        wi_dict['AssignedTo'] = format_name(wi_dict['AssignedTo'])
    main_area, sub_area = area(wi_dict['AreaPath'])
    wi_dict['main_area'] = main_area
    wi_dict['sub_area'] = sub_area
    work_items_array.append(wi_dict)

0 482379
100 571106
200 582329
300 522659
400 546235


In [57]:
pd.to_datetime(wi_dict['CreatedDate']) - today

TypeError: unsupported operand type(s) for -: 'Timestamp' and 'str'

In [52]:
pd.DataFrame(work_items_array)


,AreaPath,AssignedTo,ChangedBy,ChangedDate,CommentCount,CreatedBy,CreatedDate,IterationPath,Priority,State,...,Tags,TeamProject,Title,WorkItemType,id,import_date,main_area,rev,sub_area,weburl
0,Vienna,Aniththa Umamahesan <anumamah@microsoft.com>,Serina Kaye <serinak@microsoft.com>,2019-08-23T15:46:50.38Z,0,Aniththa Umamahesan <anumamah@microsoft.com>,2019-07-30T21:49:38.33Z,Vienna\Vibranium,2,New,...,SKUplan; WS20GA,Vienna,[AutoML] [VR] AutoML Billing & SKU,Epic,482379,2020-01-11,Vienna,5,Vienna,https://msdata.visualstudio.com/Vienna/_workit...
1,Vienna,Santhosh Pillai <sanpil@microsoft.com>,Microsoft.VisualStudio.Services.TFS <00000002-...,2020-01-08T22:01:41.773Z,1,Nishank Gupta <nigup@microsoft.com>,2019-10-09T03:29:05.707Z,Vienna\Vibranium,2,New,...,EnterpriseReadiness; Ignite2019; Tracking; WS2...,Vienna,Endpoints scenarios RBAC onboarding (Workspace...,Epic,522661,2020-01-11,Vienna,6,Vienna,https://msdata.visualstudio.com/Vienna/_workit...
2,Vienna\Assets\Assets\Metastore,John Wu <johwu@microsoft.com>,Daniel Schneider <danielsc@microsoft.com>,2020-01-11T03:44:21.317Z,0,Abe Omorogbe <osomorog@microsoft.com>,2019-11-26T21:15:53.06Z,Vienna\Manganese\03 March,2,New,...,papercut; Papercuts,Vienna,Metadata for Fast File search (Indexing file n...,Epic,552510,2020-01-11,Assets,13,Metastore,https://msdata.visualstudio.com/Vienna/_workit...
3,Vienna\Assets\Assets\Metastore,John Wu <johwu@microsoft.com>,Abe Omorogbe <osomorog@microsoft.com>,2020-01-10T21:01:48.61Z,0,Abe Omorogbe <osomorog@microsoft.com>,2019-12-12T00:44:25.753Z,Vienna\Manganese\03 March,4,New,...,papercut; Papercuts,Vienna,[CUT] Metadata for Fast File search (Searching...,Epic,565807,2020-01-11,Assets,5,Metastore,https://msdata.visualstudio.com/Vienna/_workit...
4,Vienna\Assets\Environments,Roope Astala <roastala@microsoft.com>,Chris Lauren <clauren@microsoft.com>,2020-01-08T19:33:52.913Z,1,Jordan Edwards <Jordane@microsoft.com>,2020-01-08T19:28:15.413Z,Vienna\Manganese,2,New,...,papercut,Vienna,No environment for data preparation,Bug,580752,2020-01-11,Assets,2,Environments,https://msdata.visualstudio.com/Vienna/_workit...
5,Vienna\Assets\Run History,NaN,Tzvi Keisar <tzvikei@microsoft.com>,2019-12-19T22:11:11.317Z,0,Tzvi Keisar <tzvikei@microsoft.com>,2019-12-19T22:10:46.81Z,Vienna\Manganese,2,To Do,...,Customer Engagement; Papercuts; Run History,Vienna,Naming a run would be helpful to identify them...,Task,571184,2020-01-11,Assets,2,Run History,https://msdata.visualstudio.com/Vienna/_workit...
6,Vienna\Assets\Run History,Adi Yadav <adyada@microsoft.com>,Adam Raczkowski <adamra@microsoft.com>,2020-01-03T19:50:01.463Z,1,Tzvi Keisar <tzvikei@microsoft.com>,2019-12-19T22:17:54.497Z,Vienna\Manganese,2,Done,...,Papercuts; Run History; WS2.0,Vienna,"Run history: metrics are not showing up (AUC, ...",Bug,571191,2020-01-11,Assets,3,Run History,https://msdata.visualstudio.com/Vienna/_workit...
7,Vienna\AutoML,Gaurav Singh <gasi@microsoft.com>,Sasidhar Kasturi <sasik@microsoft.com>,2020-01-10T22:16:53.927Z,1,Aditya Agarwal <adiaga@microsoft.com>,2019-12-24T01:46:36.183Z,Vienna\Vibranium\12 December,2,Approved,...,livesite; papercut,Vienna,Local run fails when trying to conitnue (Local...,Bug,572444,2020-01-11,AutoML,5,AutoML,https://msdata.visualstudio.com/Vienna/_workit...
8,Vienna\AutoML,NaN,Stephanie Lindsey <stlindse@microsoft.com>,2020-01-11T00:20:55.133Z,1,Elizabeth ♦♥ Marston <elmarsto@microsoft.com>,2019-12-06T21:51:12.26Z,Vienna\Manganese,2,New,...,WS2.0; WS20GA,Vienna,Back-forward user behaviour in AutoML reveals ...,Bug,559066,2020-01-11,AutoML,3,AutoML,https://msdata.visualstudio.com/Vienna/_workit...
9,Vienna\AutoML,Vijetha Vijayendran <vivijay@microsoft.com>,Savita Mittal <savitam@microsoft.com>,2020-01-10T22:25:39.763Z,1,Narayanan Madhu <naraymad@microsoft.com>,2019-12-10T18:38:59.133Z,Vienna\Vibranium\12 December,2,New,...,livesite; papercut,Vienna,Value Error in training when there are very fe...,Bug,564741,2020-01-11,AutoML,3,AutoML,https://msdata.visualst

In [8]:
work_items_array[0]

{'url': 'https://msdata.visualstudio.com/3adb301f-9ede-41f2-933b-fcd1a486ff7f/_apis/wit/workItems/482379',
 'id': 482379,
 'rev': 5,
 'AreaPath': 'Vienna',
 'TeamProject': 'Vienna',
 'IterationPath': 'Vienna\\Vibranium',
 'WorkItemType': 'Epic',
 'State': 'New',
 'Reason': 'New epic',
 'AssignedTo': {'displayName': 'Aniththa Umamahesan',
  'url': 'https://app.vssps.visualstudio.com/Abb9d19ad-2a17-4cfa-af94-2860838186fb/_apis/Identities/426301e4-e9c0-6c79-9edc-524e91f4e63a',
  '_links': {'avatar': {'href': 'https://msdata.visualstudio.com/_apis/GraphProfile/MemberAvatars/aad.NDI2MzAxZTQtZTljMC03Yzc5LTllZGMtNTI0ZTkxZjRlNjNh'}},
  'id': '426301e4-e9c0-6c79-9edc-524e91f4e63a',
  'uniqueName': 'anumamah@microsoft.com',
  'imageUrl': 'https://msdata.visualstudio.com/_apis/GraphProfile/MemberAvatars/aad.NDI2MzAxZTQtZTljMC03Yzc5LTllZGMtNTI0ZTkxZjRlNjNh',
  'descriptor': 'aad.NDI2MzAxZTQtZTljMC03Yzc5LTllZGMtNTI0ZTkxZjRlNjNh'},
 'CreatedDate': '2019-07-30T21:49:38.33Z',
 'CreatedBy': {'displayNa

In [ ]:
    print("running load_open")
    try:  
        os.mkdir(output_path)
    except OSError:  
        print ("Creation of the directory %s failed" % output_path)

    project = get_connstrings()


    conn = get_devopsconnection(project)

    start_time = time.time()

    print ("Getting Papercuts")
    bugids = get_query_ids(conn, get_query_papercuts())
    work_items = get_work_items(conn, bugids)
    print("Papercuts: ", len(work_items))
    print("Papercuts import time: ", time.time() - start_time)  

    from datetime import datetime
    today = datetime.utcnow().date().isoformat()

    work_items_array = []
    count = 0
    for wi in work_items:
        if (count % 100) == 0:
            print(count, wi.id)
        count+=1
        wi_dict = wi.as_dict()
        fields = wi_dict['fields']
        del wi_dict['fields']
        wi_dict.update(fields)
        wi_dict = {key.split('.')[-1]: wi_dict[key] for key in wi_dict if not key.startswith("WEF")}
        wi_dict['import_date'] = today
        wi_dict['weburl'] = f"{project['url']}/{project['id']}/_workitems/edit/{wi.id}"
        work_items_array.append(wi_dict)


    import pandas as pd
    df = pd.DataFrame(work_items_array)

    df.to_csv(output_path + '/issues-' + today + '.csv', index=False)

    df.to_csv(output_path + '/issues-latest.csv', index=False)

In [ ]:
gems = pd.read_csv(output_path + '/gems.csv')
df = pd.merge(df, gems, how='left', on=['AreaPath'])

In [ ]:
df

In [ ]:
df.to_csv(output_path + '/bugs-' + today + '.csv', index=False)
